In [1]:
import csv
import copy
import random
from collections import defaultdict

In [2]:
file1_path = "../scraped_data/related_subreddits_clean.tsv"
file2_path = "../scraped_data/related_subreddits_direct_to_file.tsv"

In [32]:
def process_file(file_path):
    """ Gets the subreddit, related subreddit pairs from file"""
    f_content = []
    with open(file_path, encoding ='utf-8') as tsvfile:
        reader = csv.reader(tsvfile, delimiter='\t')
        for row in reader:
            if len(row) >= 2:
                row[1] = row[1].split(',')[:-1]
                row[1] = [elem.split('/r/')[-1] for elem in row[1]]
            f_content.append(row)
    return f_content

In [35]:
f1_content = process_file(file1_path)
f1_content = f1_content[2:]
f2_content = process_file(file2_path)
all_content = f1_content + f2_content

In [36]:
all_count = defaultdict(int)
for node in all_content:
    all_count[node[0]] += len(node[1])

all_content_edited = []
for node in all_content:
    if all_count[node[0]] >= 2:
        all_content_edited.append(node)

print(len(all_content))
print(len(all_content_edited))
all_content = all_content_edited

45404
13534


In [37]:
random.shuffle(all_content)
subr_related_to = defaultdict(set)
for subreddit, related_reddits in all_content:
    subr_related_to[subreddit] = \
    subr_related_to[subreddit].union(set(related_reddits))
print(len(all_content))
print(len(subr_related_to))

13534
13491


In [38]:
copy_subr_related_to = copy.deepcopy(subr_related_to)
len(copy_subr_related_to)

13491

In [39]:
clusters = defaultdict(set)
count = -1
node_count = 0

def add_to_cluster(count, node):
    if node in copy_subr_related_to:
        clusters[count].add(node)
        global node_count
        node_count += 1
        
def pop_from_dict(node):
    if node in copy_subr_related_to:
        copy_subr_related_to.pop(node)

for subreddit in subr_related_to:
    count += 1
    if len(copy_subr_related_to) == 0:
        print('Breaking...')
        break
    if subreddit in copy_subr_related_to:
        add_to_cluster(count, subreddit)
        for related in copy_subr_related_to[subreddit]:
            add_to_cluster(count, related)
            for rel_to_related in copy_subr_related_to[related]:
                add_to_cluster(count, rel_to_related)
                pop_from_dict(rel_to_related)
            pop_from_dict(related)
        pop_from_dict(subreddit)

In [40]:
print(node_count)
print(len(clusters))

13942
9518


In [9]:
# clusters